## Initialize the libraries and origin and destination workspaces

In [ ]:
#This is only one time run
!pip install azure-cognitiveservices-vision-customvision

In [2]:
from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry, Region

# DESTINATION Resource Group Keys
dest_training_key = "<YOUR TRAINING KEY>"
dest_prediction_key = "<YOUR PREDICTION KEY>"
dest_project_new_name = "PowerItems_Detection_Copy"

# ORIGIN Resource Group Keys
training_key = "970bf90c200d4f429a233dee4239e8f1"
prediction_key = "23287e52d75349138403089743a504ab"

trainer = training_api.TrainingApi(training_key)
dest_trainer = training_api.TrainingApi(dest_training_key)

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection")
dest_obj_detection_domain = next(domain for domain in dest_trainer.get_domains() if domain.type == "ObjectDetection")

## Get the origin project ID reference

In [3]:
myProjects = trainer.get_projects()

In [4]:
for project in myProjects:
    print(project.name)
    print(project.id)
    print(project.description)

PowerItems_Detection_Copy
9605b559-83e4-4f6b-827e-326a790ed945

PowerItems_Detection
70e7718a-e1f1-45d5-bba1-60b224f55365
Model to detect Power line objects


In [5]:
Project = trainer.get_project(project_id="70e7718a-e1f1-45d5-bba1-60b224f55365")

## Create the destination Project 

In [7]:
dest_Project = dest_trainer.create_project(dest_project_new_name, domain_id=dest_obj_detection_domain.id)

In [8]:
for project in dest_trainer.get_projects():
    print(project.name)
    print(project.id)
    print(project.description)

PowerItems_Detection_Copy
16864536-7649-4fbb-bc03-41ca42e337c5

PowerItems_Detection
70e7718a-e1f1-45d5-bba1-60b224f55365
Model to detect Power line objects


## Get the tags on origin project and create same tags on destination project

In [9]:
dest_tags = []
for tag in trainer.get_tags(Project.id):
    dest_tags.append(dest_trainer.create_tag(dest_Project.id, tag.name))
    print(tag.name)

crossarm
pole
insulator
switch
transformer


In [10]:
dest_tags_dict = {}
dest_tag_ids = []
for tag in dest_tags:
    dest_tags_dict[tag.name] = tag.id
    dest_tag_ids.append(tag.id)

print(dest_tags_dict)
print(dest_tag_ids)

{'crossarm': '43f5a169-49c9-4add-9091-25be98fc659a', 'switch': '0e6e289e-15e1-481e-b281-1365fcee5d25', 'transformer': '3415ab3c-14af-4eb3-938a-35d32eaa8421', 'pole': '0b8bdf6e-d2c9-472f-92fa-604658912552', 'insulator': '1cabd6ae-8150-4307-a7c3-1c15b61448c9'}
['43f5a169-49c9-4add-9091-25be98fc659a', '0b8bdf6e-d2c9-472f-92fa-604658912552', '1cabd6ae-8150-4307-a7c3-1c15b61448c9', '0e6e289e-15e1-481e-b281-1365fcee5d25', '3415ab3c-14af-4eb3-938a-35d32eaa8421']


## Get the images with regions on origin project

In [11]:
tagged_images = trainer.get_tagged_images(Project.id, take=trainer.get_tagged_image_count(Project.id))

In [12]:
tagged_images_with_regions = []
for image in tagged_images:
    dest_regions = []
    dest_tags_ids = []
    
    for tag in image.tags: #for each tag on the origin image
        dest_tags_ids.append(dest_tags_dict[tag.tag_name]) 
    for region in image.regions: #for each region on the origin image
        dest_regions.append(Region(tag_id=dest_tags_dict[region.tag_name], left=region.left, top=region.top, width=region.width, height=region.height))
    
    tagged_images_with_regions.append(ImageUrlCreateEntry(url=image.image_uri, tag_ids=dest_tags_ids, regions=dest_regions))
print("Done")

Done


## Create the images with regions on destination project

In [13]:
limit = 64 # this is a limit imposed on the API, so we need to batch the creation process
count_of_images = len(tagged_images_with_regions)

for i in range(0,count_of_images,limit):
    begin=i
    end=limit+i
    if(end > count_of_images ): end = count_of_images
    dest_trainer.create_images_from_urls(dest_Project.id, images=tagged_images_with_regions[begin:end])

In [14]:
print("Count of Tagged images on origin project: " + str(trainer.get_tagged_image_count(Project.id)))
print("Count of Tagged images on destination project: " + str(dest_trainer.get_tagged_image_count(dest_Project.id)))

Count of Tagged images on origin project: 220
Count of Tagged images on destination project: 220


## Train the model on destination Project

In [15]:
import time

print ("Training... (showing status each 10 seconds)")
iteration = dest_trainer.train_project(dest_Project.id)
while (iteration.status != "Completed"):
    iteration = dest_trainer.get_iteration(dest_Project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(10)

# The iteration is now trained. Make it the default project endpoint
dest_trainer.update_iteration(dest_Project.id, iteration.id, is_default=True)
print ("Done!")

Training... (showing status each 10 seconds)
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Completed
Done!


## Get performance metrics

In [16]:
performance = dest_trainer.get_iteration_performance(dest_Project.id, iteration.id, threshold=0.15, overlap_threshold=0.30)
print("Iteration: " + iteration.name)
print("Precision: " + str(performance.precision * 100)+ "%")
print("Recall: " + str(performance.recall * 100) + "%")

Iteration: Iteration 1
Precision: 69.78417%
Recall: 44.2922384%


In [17]:
per_tag_performance = performance.per_tag_performance

In [18]:
for tag in per_tag_performance:
    print ("Tag: " + tag.name + "\t\t" + "Precision: " + str(tag.precision) + "\t" + "Recall: " + str(tag.recall))

Tag: transformer		Precision: 0.909090936	Recall: 0.8695652
Tag: crossarm		Precision: 0.6666667	Recall: 0.4117647
Tag: insulator		Precision: 0.61403507	Recall: 0.333333343
Tag: switch		Precision: 0.5882353	Recall: 0.4
Tag: pole		Precision: 0.8181818	Recall: 0.5625


## Get and use the default prediction endpoint

In [22]:
#You can make a web search online for "powerline pole" and get an URL:
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/33/240v_transformer_-_geograph.org.uk_-_1062014.jpg/170px-240v_transformer_-_geograph.org.uk_-_1062014.jpg"

<div style="width:100%; margin-left:auto; margin-right:auto; margin-bottom:5px; margin-top:17px;">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/33/240v_transformer_-_geograph.org.uk_-_1062014.jpg/170px-240v_transformer_-_geograph.org.uk_-_1062014.jpg" alt="IMAGE" /><br>
</div>

In [23]:
from azure.cognitiveservices.vision.customvision.prediction import prediction_endpoint
from azure.cognitiveservices.vision.customvision.prediction.prediction_endpoint import models

# Now there is a trained endpoint that can be used to make a prediction
dest_predictor = prediction_endpoint.PredictionEndpoint(dest_prediction_key)

# Open the sample image and get back the prediction results.
results = dest_predictor.predict_image_url(dest_Project.id, iteration_id=iteration.id, url=url)

In [25]:
# Display the results.
for prediction in results.predictions:
    if(prediction.probability > 0.2): # only show the prediction with more than 20% threshold probability
        print ("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100), "   Box relative points:",prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height)

	insulator: 55.09%    Box relative points: 0.9736056 0.116215907 0.0263844132 0.053531
	insulator: 68.11%    Box relative points: 0.08418168 0.1291371 0.0814568549 0.0867869556
	transformer: 72.06%    Box relative points: 0.415163636 0.6361095 0.2531705 0.291148365
